In [ ]:
import pandas as pd # Librería para la manipulación y el análisis de datos
import numpy as np # Librería para la manipulación de datos y para la ejecución de operaciones matemáticas
import matplotlib.pyplot as plt # Librería para la visualización de datos
import seaborn as sns # Librería para la visualización de datos
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder # Librería para crear modelos de ML
import os
# conectar con drive
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/data/TFM/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocesamiento de datasets

## DATASET 'CUSTUMER_COMERCIAL_ACTIVITY.CSV'

In [ ]:
cca = pd.read_csv(os.path.join(data_path, 'customer_commercial_activity.csv'),index_col=0)
cca.sample(3)

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
2354786,1305508.0,2018-04,2017-09,KHE,0.0,03 - UNIVERSITARIO
1117090,1006578.0,2018-02,2015-02,KHD,1.0,03 - UNIVERSITARIO
2584961,1105355.0,2018-05,2015-11,KHE,0.0,03 - UNIVERSITARIO


In [ ]:
cca['entry_date'].describe()

,entry_date
count,1698040
unique,43
top,2017-10
freq,120572


#### Tratamos cada una de las 6 columnas:

In [ ]:
# pk_cid: Identificador de cliente

cca['pk_cid'].nunique()
cca['pk_cid'].describe()
cca['pk_cid'].isnull().sum()

np.int64(1)

In [ ]:
# pk_partition: Identificador de partición. Mensualmente se ingesta en la tabla una partición con el estado de la base de clientes

# Cambiamos a datetime
cca['pk_partition'] = pd.to_datetime(cca['pk_partition'])
cca['pk_partition'].isnull().sum() # 0

np.int64(1)

In [ ]:
# entry_date: Fecha en la que se realizó la primera contratación a través de EasyMoney

# Cambiamos a datetime
cca['entry_date'] = pd.to_datetime(cca['entry_date'])
cca['entry_date'].isnull().sum() # 0


np.int64(1)

In [ ]:
# Crear un campo booleano para saber si el cliente se ha unido hace menos de un año

last_date = cca['entry_date'].max()
cca['new_customer'] = (last_date - cca['entry_date']).dt.days < 365
cca['new_customer'] = cca['new_customer'].astype(int)

In [ ]:
cca.new_customer.value_counts()

,count
new_customer,
0,1267095
1,430946


In [ ]:
# entry_channel: Canal de captación del cliente

cca['entry_channel'].nunique() # 68
cca['entry_channel'].isnull().sum() # 133033
cca['entry_channel'].value_counts(dropna=False)

,count
entry_channel,
KHE,1105543
KFC,282716
KAT,146907
KHK,58106
KFA,28672
...,...
KBU,1
KAI,1
KCK,1


In [ ]:
# Imputamos nulos con la moda
cca['entry_channel'] = cca['entry_channel'].fillna(cca['entry_channel'].mode()[0])

In [ ]:
# active_customer: Indicador de actividad del cliente en la app EasyMoney

cca['active_customer'].nunique() # 2
cca['active_customer'].isnull().sum() # 0
cca['active_customer'].value_counts(dropna=False)

,count
active_customer,
0.0,927387
1.0,770653
NaN,1


In [ ]:
# --- segment ---

cca['segment'].nunique() # 3
cca['segment'].isnull().sum() # 133944
pd.merge(cca['segment'].value_counts(dropna=False, normalize=True), cca['segment'].value_counts(dropna=False), left_index=True, right_index=True)

,proportion,count
segment,,
03 - UNIVERSITARIO,0.691128,1173564
02 - PARTICULARES,0.290445,493187
01 - TOP,0.018238,30969
NaN,0.000189,321


In [ ]:
# Imputamos por moda tambien
cca['segment'] = cca['segment'].fillna(cca['segment'].mode()[0])

In [ ]:
# la fila 448 de cca tiene nulos

cca[cca.isnull().any(axis=1)]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,new_customer
448,NaN,NaT,NaT,KHE,NaN,03 - UNIVERSITARIO,0


In [ ]:
# eliminar la fila con los 4 nulos

cca = cca.dropna()

In [ ]:
cca

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,new_customer
0,1375586.0,2018-01-01,2018-01-01,KHL,1.0,02 - PARTICULARES,1
1,1050611.0,2018-01-01,2015-08-01,KHE,0.0,03 - UNIVERSITARIO,0
2,1050612.0,2018-01-01,2015-08-01,KHE,0.0,03 - UNIVERSITARIO,0
3,1050613.0,2018-01-01,2015-08-01,KHD,0.0,03 - UNIVERSITARIO,0
4,1050614.0,2018-01-01,2015-08-01,KHE,1.0,03 - UNIVERSITARIO,0
...,...,...,...,...,...,...,...
4488231,1261771.0,2018-07-01,2017-06-01,KFC,1.0,02 - PARTICULARES,0
4488232,1261769.0,2018-07-01,2017-06-01,KFC,0.0,02 - PARTICULARES,0
4488233,1261766.0,2018-07-01,2017-06-01,KFC,0.0,02 - PARTICULARES,0
4488234,1261764.0,2018-07-01,2017-06-01,KHN,1.0,01 - TOP,0


## DATASET 'COSTUMER_PRODUCTS.CSV'

In [ ]:
df_customer_products = pd.read_csv(os.path.join(data_path, 'customer_products.csv'), index_col = 0)
df_customer_products

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1050611,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1050612,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1050613,2018-01,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1050614,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5048404,1427926,2018-08,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5048405,1427938,2018-08,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5048406,1427936,2018-08,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5048407,1427935,2018-08,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
df_customer_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1961966 entries, 0 to 5048408
Data columns (total 17 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   pk_partition        object 
 2   short_term_deposit  int64  
 3   loans               int64  
 4   mortgage            int64  
 5   funds               int64  
 6   securities          int64  
 7   long_term_deposit   int64  
 8   em_account_pp       int64  
 9   credit_card         int64  
 10  payroll             float64
 11  pension_plan        float64
 12  payroll_account     float64
 13  emc_account         float64
 14  debit_card          float64
 15  em_account_p        float64
 16  em_acount           float64
dtypes: float64(7), int64(9), object(1)
memory usage: 269.4+ MB


In [ ]:
# Convertir 'pk_partition' a un formato datetime

df_customer_products['pk_partition'] = pd.to_datetime(df_customer_products['pk_partition'], format='%Y-%m')

# Pandas muestra el día 01 de cada mes por defecto, pero no supone una complicación en ningún sentido.

In [ ]:
df_customer_products.head()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1050611,2018-01-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1050612,2018-01-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1050613,2018-01-01,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1050614,2018-01-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
df_customer_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1961966 entries, 0 to 5048408
Data columns (total 17 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int64         
 1   pk_partition        datetime64[ns]
 2   short_term_deposit  int64         
 3   loans               int64         
 4   mortgage            int64         
 5   funds               int64         
 6   securities          int64         
 7   long_term_deposit   int64         
 8   em_account_pp       int64         
 9   credit_card         int64         
 10  payroll             float64       
 11  pension_plan        float64       
 12  payroll_account     float64       
 13  emc_account         float64       
 14  debit_card          float64       
 15  em_account_p        float64       
 16  em_acount           float64       
dtypes: datetime64[ns](1), float64(7), int64(9)
memory usage: 269.4 MB


In [ ]:
df_customer_products.describe()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
count,1.961966e+06,1961966,1.961966e+06,1.961966e+06,1.961966e+06,1.961966e+06,1.961966e+06,1.961966e+06,1961966.0,1.961966e+06,1.961905e+06,1.961904e+06,1.961965e+06,1.961965e+06,1.961965e+06,1.961965e+06,1.961965e+06
mean,1.194568e+06,2018-04-15 07:33:13.396012032,5.518444e-03,8.919624e-05,6.575037e-05,3.917499e-03,3.805367e-03,2.014918e-02,0.0,1.409810e-02,3.800796e-02,4.013499e-02,5.892103e-02,6.561687e-02,1.040860e-01,7.645396e-06,8.419029e-01
min,1.589100e+04,2018-01-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.093964e+06,2018-03-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,1.198618e+06,2018-05-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.301352e+06,2018-06-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,1.440461e+06,2018-08-01 00:00:00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
std,1.370237e+05,NaN,7.408100e-02,9.443958e-03,8.108396e-03,6.246723e-02,6.157019e-02,1.405105e-01,0.0,1.178955e-01,1.912155e-01,1.962758e-01,2.354769e-01,2.476112e-01,3.053721e-01,2.765021e-03,3.648321e-01


In [ ]:
# pk_partition (existe en 3 de los los datasets del tfm), muestra una actualización mensual
# a partir de la cual se puede ver en las demás columnas si los clientes han estado activos o no.

In [ ]:
# Ejemplo con cliente '1375586'.

df_customer_products[df_customer_products['pk_cid'] == 1375586]

# No queda claro si la descripcion de las variables es correcta, porque pone que este cliente tuvo 'tenencia de tarjeta de débito'
# durante unos pocos meses, pero otros no. Se refiere a uso y no tenencia?

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1047196,1375586,2018-02-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1681276,1375586,2018-03-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2299297,1375586,2018-04-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2714121,1375586,2018-05-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3343336,1375586,2018-06-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4581564,1375586,2018-07-01,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
df_customer_products.duplicated().sum()

np.int64(0)

In [ ]:
df_customer_products.isnull().sum()

,0
pk_cid,0
pk_partition,0
short_term_deposit,0
loans,0
mortgage,0
funds,0
securities,0
long_term_deposit,0
em_account_pp,0
credit_card,0


In [ ]:
# Comprobación individual de columnas con nulos

for col in df_customer_products.columns:
  if df_customer_products[col].isnull().sum() > 0:
    print(df_customer_products[col].value_counts(dropna=False))
    print()
    print(df_customer_products[col].value_counts(normalize=True, dropna=False) * 100) # visualizacion en porcentajes
    print()

payroll
0.0    1887337
1.0      74568
NaN         61
Name: count, dtype: int64

payroll
0.0    96.196213
1.0     3.800677
NaN     0.003109
Name: proportion, dtype: float64

pension_plan
0.0    1883163
1.0      78741
NaN         62
Name: count, dtype: int64

pension_plan
0.0    95.983468
1.0     4.013372
NaN     0.003160
Name: proportion, dtype: float64

payroll_account
0.0    1846364
1.0     115601
NaN          1
Name: count, dtype: int64

payroll_account
0.0    94.107849
1.0     5.892100
NaN     0.000051
Name: proportion, dtype: float64

emc_account
0.0    1833227
1.0     128738
NaN          1
Name: count, dtype: int64

emc_account
0.0    93.438265
1.0     6.561684
NaN     0.000051
Name: proportion, dtype: float64

debit_card
0.0    1757752
1.0     204213
NaN          1
Name: count, dtype: int64

debit_card
0.0    89.591359
1.0    10.408590
NaN     0.000051
Name: proportion, dtype: float64

em_account_p
0.0    1961950
1.0         15
NaN          1
Name: count, dtype: int64

em_account

In [ ]:
# Imputación de nulos por la moda

for col in df_customer_products.columns:

  if df_customer_products[col].isnull().sum() > 0:

    valor_moda = df_customer_products[col].mode()[0]
    df_customer_products[col].fillna(valor_moda, inplace=True)

/tmp/ipython-input-4020819241.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_customer_products[col].fillna(valor_moda, inplace=True)


In [ ]:
df_customer_products[['payroll', 'pension_plan']].isnull().sum()

,0
payroll,0
pension_plan,0


In [ ]:
# Visualizacion como integers para que dataset sea mas uniforme

for col in df_customer_products.columns:
  if df_customer_products[col].dtype == 'float64':
    df_customer_products[col] = df_customer_products[col].astype('int64')

In [ ]:
df_customer_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1961966 entries, 0 to 5048408
Data columns (total 17 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int64         
 1   pk_partition        datetime64[ns]
 2   short_term_deposit  int64         
 3   loans               int64         
 4   mortgage            int64         
 5   funds               int64         
 6   securities          int64         
 7   long_term_deposit   int64         
 8   em_account_pp       int64         
 9   credit_card         int64         
 10  payroll             int64         
 11  pension_plan        int64         
 12  payroll_account     int64         
 13  emc_account         int64         
 14  debit_card          int64         
 15  em_account_p        int64         
 16  em_acount           int64         
dtypes: datetime64[ns](1), int64(16)
memory usage: 269.4 MB


### Análisis univariable

In [ ]:
for col in df_customer_products.columns:
    print(df_customer_products[col].value_counts())
    print()

pk_cid
1269579    8
1269580    8
1269582    8
1269583    8
1269585    8
          ..
1427915    1
1427916    1
1427917    1
1427918    1
1427919    1
Name: count, Length: 351439, dtype: int64

pk_partition
2018-07-01    339339
2018-06-01    252104
2018-05-01    249926
2018-04-01    247463
2018-03-01    245258
2018-02-01    242521
2018-01-01    239493
2018-08-01    145862
Name: count, dtype: int64

short_term_deposit
0    1951139
1      10827
Name: count, dtype: int64

loans
0    1961791
1        175
Name: count, dtype: int64

mortgage
0    1961837
1        129
Name: count, dtype: int64

funds
0    1954280
1       7686
Name: count, dtype: int64

securities
0    1954500
1       7466
Name: count, dtype: int64

long_term_deposit
0    1922434
1      39532
Name: count, dtype: int64

em_account_pp
0    1961966
Name: count, dtype: int64

credit_card
0    1934306
1      27660
Name: count, dtype: int64

payroll
0    1887398
1      74568
Name: count, dtype: int64

pension_plan
0    1883225
1     

## DATASET 'CUSTOMER_SOCIODEMOGRAPHICS.CSV'

In [ ]:
sociodemographics = pd.read_csv(os.path.join(data_path, 'customer_sociodemographics.csv'),index_col=0)
sociodemographics

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01,ES,29.0,H,35.0,N,87218.10
1,1050611,2018-01,ES,13.0,V,23.0,N,35548.74
2,1050612,2018-01,ES,13.0,V,23.0,N,122179.11
3,1050613,2018-01,ES,50.0,H,22.0,N,119775.54
4,1050614,2018-01,ES,50.0,V,23.0,N,NaN
...,...,...,...,...,...,...,...,...
8319379,1274601,2018-12,ES,16.0,H,22.0,N,82209.24
8319380,1274602,2018-12,ES,25.0,V,21.0,N,15878.04
8319381,1274603,2018-12,ES,43.0,V,25.0,N,231297.30
8319382,1274604,2018-12,ES,8.0,V,24.0,N,64542.45


#### Tratamos cada una de las 8 columnas:

In [ ]:
# pk_cid: Identificador de cliente

sociodemographics['pk_cid'].isnull().sum() # 0
sociodemographics['pk_cid'].nunique() # 456373 clientes diferentes

421121

In [ ]:
# pk_partition: Identificador de partición. Mensualmente se ingesta en la tabla una partición con el estado de la base de clientes

# Pasamos a tipo de dato datetime
sociodemographics['pk_partition'] = pd.to_datetime(sociodemographics['pk_partition'])
sociodemographics['pk_partition'].isnull().sum() # 0

np.int64(0)

In [ ]:
# deceased: Indicador de fallecimiento

# 0.000018% fallecidos, los eliminamos para simplificar el dataset y nos quitamos la columna de encima
sociodemographics['deceased'].value_counts(dropna=False, normalize=True)
sociodemographics = sociodemographics[sociodemographics['deceased'] == 'N'].drop(columns = 'deceased', axis=0)

In [ ]:
# gender: Sexo del cliente

# Imputamos los nulos con la moda
sociodemographics['gender'].value_counts(dropna=False)  # 25 nulos
sociodemographics['gender'] = sociodemographics['gender'].fillna(sociodemographics['gender'].mode()[0])

In [ ]:
# region_code: Provincia de residencia del cliente (para ES)

sociodemographics['region_code'].nunique() # 52
sociodemographics['region_code'].isnull().sum() # 2264: clientes que no viven en España.
sociodemographics[sociodemographics['country_id'] == 'ES']['region_code'].isnull().sum()
# No hay nulos que tengan countryID en españa


np.int64(11)

In [ ]:
sociodemographics['region_code'].nunique()

52

In [ ]:
# Imputamos estos nulos fuera de españa con el valor 00
sociodemographics['region_code'] = sociodemographics['region_code'].fillna(0) # Los imputamos con un nuevo valor 00 (fuera de ESpaña)
sociodemographics['region_code'].isnull().sum() # 0

np.int64(0)

In [ ]:
# salary: Ingresos brutos de la unidad familiar

sociodemographics['salary'].isnull().sum() # 1541104

np.int64(724820)

In [ ]:
# Vemos que los menores de 18 tienen salario NaN, los imputamos con 0
sociodemographics[sociodemographics['age']<19].groupby(['age'])['salary'].mean()

sociodemographics.loc[sociodemographics['age'] < 18, 'salary'] = 0
sociodemographics['salary'].isnull().sum()

np.int64(703445)

In [ ]:
# El resto de nulos los imputamos con la media de cada grupo de region_code y edad
salario_sin_nulos = sociodemographics.groupby(['region_code', 'age'])['salary'].transform(lambda x: x.fillna(x.mean()))
salario_sin_nulos.isnull().sum() # 69858 nulos

# Como quedan nulos, los imputamos con la media global
salario_sin_nulos = salario_sin_nulos.fillna(salario_sin_nulos.mean())
sociodemographics['salary'] = salario_sin_nulos
sociodemographics['salary'].isnull().sum()  # 0

np.int64(0)

In [ ]:
# country_id: Pais de residencia del cliente

sociodemographics['country_id'].isnull().sum() # 0
sociodemographics['country_id'].nunique() # 41
sociodemographics['country_id'].value_counts(dropna=False, normalize=True).head(5)

,proportion
country_id,
ES,0.999601
GB,0.000074
FR,0.000042
DE,0.000036
CH,0.000036


In [ ]:
# Eliminamos los que son fuera de España ya que representan menos de un 0.1% y nos quitamos la columna de encima

sociodemographics = sociodemographics[sociodemographics['country_id'] == 'ES']
sociodemographics = sociodemographics.drop(columns = 'country_id', axis=1)

In [ ]:
# age: Edad del cliente

sociodemographics['age'].isnull().sum() # 0
sociodemographics['age'].value_counts(dropna=False).sort_index()

,count
age,
2.0,356
3.0,870
4.0,1417
5.0,1840
6.0,1856
...,...
101.0,46
102.0,19
103.0,13


## DATASET 'SALES.CSV'

In [ ]:
sales = pd.read_csv(os.path.join(data_path, 'sales.csv'),index_col=0)
sales

,pk_sale,cid,month_sale,product_ID,net_margin
0,6666,33620,2018-05-01,2335,952.9
1,6667,35063,2018-06-01,2335,1625.2
2,6668,37299,2018-02-01,2335,1279.7
3,6669,39997,2018-02-01,2335,1511.9
4,6670,44012,2018-02-01,2335,1680.3
...,...,...,...,...,...
240768,247434,1553456,2019-05-01,4657,56.7
240769,247435,1553541,2019-05-01,4657,66.5
240770,247436,1553559,2019-05-01,4657,73.0
240771,247437,1553565,2019-05-01,4657,82.3


In [ ]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240773 entries, 0 to 240772
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   pk_sale     240773 non-null  int64  
 1   cid         240773 non-null  int64  
 2   month_sale  240773 non-null  object 
 3   product_ID  240773 non-null  int64  
 4   net_margin  240773 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 11.0+ MB


#### Tratamos cada una de las 5 columnas:

In [ ]:
# pk_sale: Identificador de venta

sales['pk_sale'].nunique()
sales['pk_sale'].isnull().sum() # 0

np.int64(0)

In [ ]:
# cid: Identificador de cliente

sales['cid'].nunique()
sales['cid'].isnull().sum() # 0

np.int64(0)

In [ ]:
# month_sale: Mes de venta. Mes en el que se contrata un producto

# Pasamos a datetime
sales['month_sale'] = pd.to_datetime(sales['month_sale'])
sales['month_sale'].isnull().sum() # 0

np.int64(0)

In [ ]:
# product_ID: Identificador de producto

sales['product_ID'].nunique() # 13
sales['product_ID'].isnull().sum() # 0
sales['product_ID'].value_counts(dropna=False)

,count
product_ID,
4657,117435
3819,38840
1364,19369
2234,18750
2235,18110
8871,12323
2312,5206
2335,4441
2336,4227


In [ ]:
# net_margin: Margen neto generado por la venta ($)

sales['net_margin'].isnull().sum() # 0
sales['net_margin'].describe() # Detectamos un outlayer de 18466.4

,net_margin
count,240773.000000
mean,606.878164
std,1735.806089
min,25.000000
25%,61.400000
50%,69.100000
75%,79.200000
max,18466.400000


In [ ]:
# A que products_ID diferentes pertenecen los sales con un 'net_margin' superior a 10000

sales[sales['net_margin'] > 10000]['product_ID'].value_counts()
# Vemos que el producto 1364 es el mas caro con lo que no es un outlier directamente

,count
product_ID,
1364,1043


## DATASET 'PRODUCT_DESCRIPTION.CSV'

In [ ]:
product_description = pd.read_csv(os.path.join(data_path, 'product_description.csv'),index_col=0)
product_description

,pk_product_ID,product_desc,family_product
0,4657,em_acount,account
1,3819,debit_card,payment_card
2,1364,pension_plan,pension_plan
3,2234,payroll,account
4,2235,payroll_account,account
5,8871,emc_account,account
6,2312,credit_card,payment_card
7,2335,short_term_deposit,investment
8,2336,long_term_deposit,investment
9,2673,securities,investment


## Datasets limpios

In [ ]:
# Pasar a csv todos los dataframes limpios
cca.to_csv(os.path.join(data_path, 'clean_datasets/customer_commercial_activity_clean.csv'))
df_customer_products.to_csv(os.path.join(data_path, 'clean_datasets/customer_products_clean.csv'))
sociodemographics.to_csv(os.path.join(data_path, 'clean_datasets/customer_sociodemographics_clean.csv'))
sales.to_csv(os.path.join(data_path, 'clean_datasets/sales_clean.csv'))
product_description.to_csv(os.path.join(data_path, 'clean_datasets/product_description_clean.csv'))